In [13]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests as rq
from github import Github, GithubException
import time
from shutil import copyfile
from pymongo import MongoClient
import re

In [24]:
client = MongoClient()
db = client['repos-database']
dockers = db['docker_repos']
post_id = posts.insert_one(post).inserted_id
post_id

In [35]:
headers = {'user-agent':'Mozilla/5.0'}

credentials = Github("user", "password")

search = credentials.search_code("filename:dockerfile")
print(len(list(search.get_page(0))))
print(search.totalCount)

print(size)
page_n = 0
while(page_n<35):
    i=1
    try:
        results = list(search.get_page(page_n))
        files=[]
        previousDockerLine = ""
        for file in results:
            print(page_n*10+i)
            i+=1
            dockerFrom = [] 
            dockerRun = [] 
            dockerAdd = [] 
            dockerWorkdir = [] 
            dockerExpose = [] 
            dockerCopy = [] 
            dockerEntrypoint = [] 
            dockerCmd = [] 
            dockerVolume = [] 
            dockerUser = [] 
            dockerLabel = []
            dockerArg = []
            dockerEnv = []
            dockerOnbuild = []
            dockerMaintainer =""
            page = rq.get(file.html_url,headers=headers)
            parsed_page = BeautifulSoup(page.text,"html.parser").find("table",attrs={'class':'highlight tab-size js-file-line-container'})
            dockerfile = ""
            if(parsed_page!=None and parsed_page.find_all("td",attrs={'class':'blob-code blob-code-inner js-file-line'})!=None):
                for td in parsed_page.find_all("td",attrs={'class':'blob-code blob-code-inner js-file-line'}):
                    if(td.text.lower().startswith("from ")):
                        dockerFrom.append(re.match('\w{4} (.*)',td.text).group(1))
                        previousDockerLine = "FROM"
                    elif(td.text.lower().startswith("run ")):
                        dockerRun.append(re.match('\w{3} (.*)',td.text).group(1))
                        previousDockerLine = "RUN"
                    elif(td.text.lower().startswith("add ")):
                        dockerAdd.append(re.match('\w{3} (.*)',td.text).group(1))
                        previousDockerLine = "ADD"
                    elif(td.text.lower().startswith("workdir ")):
                        dockerWorkdir.append(re.match('\w{7} (.*)',td.text).group(1))
                        previousDockerLine = "WORKDIR"
                    elif(td.text.lower().startswith("expose ")):
                        dockerExpose.append(re.match('\w{6} (.*)',td.text).group(1))
                        previousDockerLine = "EXPOSE"
                    elif(td.text.lower().startswith("copy ")):
                        dockerCopy.append(re.match('\w{4} (.*)',td.text).group(1))
                        previousDockerLine = "COPY"
                    elif(td.text.lower().startswith("entrypoint ")):
                        dockerEntrypoint.append(re.match('\w{10} (.*)',td.text).group(1))
                        previousDockerLine = "ENTRYPOINT"
                    elif(td.text.lower().startswith("cmd ")):
                        dockerCmd.append(re.match('\w{3} (.*)',td.text).group(1))
                        previousDockerLine = "CMD"
                    elif(td.text.lower().startswith("volume ")):
                        dockerVolume.append(re.match('\w{6} (.*)',td.text).group(1))
                        previousDockerLine = "VOLUME"
                    elif(td.text.lower().startswith("user ")):
                        dockerUser.append(re.match('\w{4} (.*)',td.text).group(1))
                        previousDockerLine = "USER"
                    elif(td.text.lower().startswith("label ")):
                        dockerLabel.append(re.match('\w{5} (.*)',td.text).group(1))
                        previousDockerLine = "LABEL"
                    elif(td.text.lower().startswith("arg ")):
                        dockerArg.append(re.match('\w{3} (.*)',td.text).group(1))
                        previousDockerLine = "ARG"
                    elif(td.text.lower().startswith("env ")):
                        dockerEnv.append(re.match('\w{3} (.*)',td.text).group(1))
                        previousDockerLine = "ENV"
                    elif(td.text.lower().startswith("onbuild ")):
                        dockerOnbuild.append(re.match('\w{7} (.*)',td.text).group(1))
                        previousDockerLine = "ONBUILD"
                    elif(td.text.lower().startswith("maintainer ")):
                        dockerMaintainer = re.match('\w{10} (.*)',td.text).group(1)
                        previousDockerLine = "MAINTAINER"
                    elif(not td.text.strip()):
                        previousDockerLine = ""
                    elif(previousDockerLine == "RUN"):
                        dockerRun.append(td.text)
                    elif(previousDockerLine == "LABEL"):
                        dockerLabel.append(td.text)
            page = rq.get(file.html_url,headers=headers)
            code_link = BeautifulSoup(page.text,"html.parser").find("a",attrs={'data-hotkey':'g c'})
            page = rq.get("https://github.com"+code_link.get('href'),headers=headers)
            language = BeautifulSoup(page.text,"html.parser").find("ol",attrs={'class':'repository-lang-stats-numbers'})
            if(language!=None):
                languages = language.find_all("a")
                language = []
                for l in languages:
                    language.append(l.find("span",attrs={'class':'lang'}).text+" "+l.find("span",attrs={'class':'percent'}).text)                          
            repo_name = file.repository.full_name
            repo_date = file.repository.created_at
            repo_last_updated = file.repository.updated_at
            '''if(not file.repository.organization==None):
                repo_organization = file.repository.organization
            else:
                repo_organization = None'''
            dockerfile = dict(zip(['FROM','MAINTAINER','RUN','ADD','WORKDIR','EXPOSE','COPY','ENTRYPOINT','CMD','VOLUME','USER','LABEL','ARG','ENV','ONBUILD'],\
                                 [dockerFrom,dockerMaintainer,dockerRun,dockerAdd,dockerWorkdir,dockerExpose,dockerCopy,dockerEntrypoint,dockerCmd,dockerVolume,dockerUser,dockerLabel,dockerArg,dockerEnv,dockerOnbuild]))
            #file.append(dict(zip(['Repositório','Organização','Dockerfile','Linguagens','Data Criação','Data Update','Repo link'],\
            #                     [repo_name,repo_organization,dockerfile,language,repo_date,repo_last_updated,"https://github.com"+code_link.get('href')])))
            docker_id = dockers.insert_one(dict(zip(['Repositório','Dockerfile','Linguagens','Data Criação','Data Update','Repo link'],\
                                 [repo_name,dockerfile,language,repo_date,repo_last_updated,"https://github.com"+code_link.get('href')]))).inserted_id
        '''dataDockers = pd.DataFrame(files,\
                                       columns=['Repositório','Organização','Dockerfile',
                                                'Linguagens','Data Criação','Data Update','Repo link'])
        if(page_n==0):
            dataDockers.to_csv('dockers.csv',mode='a')
        else:
            dataDockers.to_csv('dockers.csv',mode='a',header=False)
        '''
        page_n+=1
    except GithubException:
        print("Extourou as requisições por minuto, irá esperar um minuto para continuar")
        time.sleep(60)
#copyfile('dockers.csv', 'csv/dockers.csv')

30
1051669
35048.833333333336
Extourou as requisições por minuto, irá esperar um minuto para continuar


KeyboardInterrupt: 